# Customer Segmentation

본 대회의 주제는 이커머스 기업의 **고객을 세분화**하여 현재 기업의 상황을 판단하고, 해당 솔루션에 맞는 솔루션을 제시하는 것입니다.  
이를 위해 베이스라인에서는 기본적인 EDA를 수행한 후, 여러가지 방법으로 고객을 분류하겠습니다. 

### 라이브러리 불러오기(시각화 및 한글폰트) 

베이스라인을 위해 필요한 라이브러리들을 불러옵니다. 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import datetime as dt
import numpy as np
# 한글 폰트 설정하기
#fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
#fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='AppleGothic')

### 데이터 로드

대회 데이터를 불러옵니다. 

In [2]:
online_sales = pd.read_csv('Onlinesales_info.csv')
customers_data = pd.read_csv('Customer_info.csv')
discount_coupon = pd.read_csv('Discount_info.csv')
marketing_spend = pd.read_csv('Marketing_info.csv')
tax_amount = pd.read_csv('Tax_info.csv')

In [3]:
#total_data = 
online_sales.head(5)

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used


In [5]:

customers_data.head(5)

,고객ID,성별,고객지역,가입기간
0,USER_1358,남,Chicago,12
1,USER_0190,남,California,43
2,USER_0066,남,Chicago,33
3,USER_0345,여,California,30
4,USER_0683,남,California,49


In [6]:
discount_coupon.head(5)

,월,제품카테고리,쿠폰코드,할인율
0,Jan,Apparel,SALE10,10
1,Feb,Apparel,SALE20,20
2,Mar,Apparel,SALE30,30
3,Jan,Nest-USA,ELEC10,10
4,Feb,Nest-USA,ELEC20,20


In [7]:
marketing_spend.head(5)

,날짜,오프라인비용,온라인비용
0,2019-01-01,4500,2424.50
1,2019-01-02,4500,3480.36
2,2019-01-03,4500,1576.38
3,2019-01-04,4500,2928.55
4,2019-01-05,4500,4055.30


In [8]:
tax_amount.head(5)

,제품카테고리,GST
0,Nest-USA,0.10
1,Office,0.10
2,Apparel,0.18
3,Bags,0.18
4,Drinkware,0.18


In [ ]:
df_total = pd.merge(online_sales, customers_data,on='고객ID')

In [1]:

import pandas as pd

import sweetviz as sv

In [8]:
advert_report = sv.analyze(tax_amount)
#display the report
advert_report.show_html('./tax_amount.html')

                                             |          | [  0%]   00:00 -> (? left)

Report ./tax_amount.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
online_sales.head(5)

In [ ]:
online_sales.describe()

In [ ]:
online_sales.info()

In [ ]:
online_sales.isnull().sum()

In [ ]:
for c in online_sales.columns:
    print(online_sales[c].value_counts())

In [ ]:
online_sales['고객ID'].value_counts().reset_index().plot(kind='bar')

In [ ]:
online_sales['제품ID'].value_counts().reset_index().plot(kind='bar')

In [ ]:
coupon_transaction = online_sales[['쿠폰상태','거래ID', '거래날짜']]
coupon_transaction

In [ ]:
online_sales['쿠폰상태'].value_counts().plot(kind='pie')


In [ ]:
coupon_transaction = coupon_transaction.assign(Month=pd.to_datetime(online_sales['거래날짜']).dt.to_period('M'))
#coupon_transaction['쿠폰상태'] = coupon_transaction['쿠폰상태'].replace('Used', 1).replace('Not Used',2).replace('Clicked',3)

coupon_transaction = coupon_transaction[['쿠폰상태', 'Month']]
coupon_transaction = coupon_transaction.groupby('Month')['쿠폰상태'].value_counts()

coupon_transaction=coupon_transaction.sort_index(level=0)

In [ ]:
from itertools import cycle, islice

my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k']), None, 3))

coupon_transaction.plot(kind='bar',  stacked=True, color=my_colors, figsize=(10,6))

In [ ]:
coupon_transaction

In [ ]:
i=0
lst = []
res = []
for co in coupon_transaction.reset_index(drop=True):
    if  i>2 and i%3==0 :
        res.append(lst[2]/lst[0])
        lst = []
    lst.append(co)
    i +=1
res.append(lst[2]/lst[0]) 
    

In [ ]:
res

In [ ]:
res = list(map(lambda x :round(x,4)-0.5,res))

In [ ]:
import matplotlib.pyplot as plt
ax = sns.barplot(x=np.arange(len(res))+1, y=res)
ax.bar_label(ax.containers[0], labels=[f'{e+0.5:.2f}' for e in res])
ax.set_xlabel('월별 클릭 대비 쿠폰 적용율')
#plt.axis('off')
plt.show()

In [ ]:
#3,6,9,12월에 발급한 쿠폰 할인율이 30%으로 높았고 Marketing비용이 11,12월에 높았으나 그 효과가 미비하고, 여름7,8월의 쿠폰적용률이 높은 것으로 보아 연초부터 시작한 쿠폰 이벤트의 효과가 여름에 적용된 것으로 보임.
#또한, 클릭대비 실제 사용률은 9월이 가장 높았다.

In [ ]:
customers_data.head(5)

In [ ]:
discount_coupon.head(5)

In [ ]:
category_list = discount_coupon['제품카테고리'].unique()
for cl in category_list:
    discount_coupon[discount_coupon['제품카테고리']==cl].set_index('월').plot(kind='bar')    


In [ ]:
marketing_spend.head(5)

In [ ]:
marketing_spend.info()

In [ ]:
marketing_spend.describe()

In [ ]:
marketing_spend.head(-5)

In [ ]:

#counts_by_month = online_sales.assign(Month=pd.to_datetime(online_sales['거래날짜']).dt.to_period('M'))['Month'].value_counts().sort_index()
marketing_spend= marketing_spend.assign(Month=pd.to_datetime(marketing_spend['날짜']).dt.to_period('M'))
marketing_spend.groupby('Month')['온라인비용'].sum().plot(kind='bar')

In [ ]:
marketing_spend.groupby('Month')['오프라인비용'].sum().plot(kind='bar')

In [ ]:
a = marketing_spend.groupby('Month')['온라인비용'].sum().to_frame().reset_index()
b = marketing_spend.groupby('Month')['오프라인비용'].sum().to_frame().reset_index()
a["온라인비용"]=a["온라인비용"].astype('int')
ax_list  = a.Month.astype(str).reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
weight_counts = {
    
    "Offline": np.array(b['오프라인비용']),
    "Online": np.array(a['온라인비용']),
}
width = 0.5

fig, ax = plt.subplots()
fig.set_size_inches(13, fig.get_figheight(), forward=True)

bottom = np.zeros(12)
for boolean, weight_count in weight_counts.items():
    print(boolean, weight_count)
for boolean, weight_count in weight_counts.items():
    p = ax.bar(ax_list, weight_count , width, label=boolean, bottom=bottom)
    bottom += weight_count

ax.set_title("Marketing Cost")
ax.legend(loc="upper center")
plt.show()

In [ ]:
tax_amount.head(5)

In [ ]:
customers_data


In [ ]:
customer_regions = customers_data['고객지역'].value_counts().reset_index().set_index('고객지역')

customer_regions

In [ ]:
online_sales

In [ ]:
#카테고리 판매횟수
category_sales= online_sales['제품카테고리'].value_counts()


In [ ]:
category_sales.sort_values().plot(kind='barh')

In [ ]:
category_sales

In [ ]:
category_sales.index[0]

In [ ]:
# 카테고리 판매횟수 상위
category_sales[:5].plot(kind='bar', y = 'count')


In [ ]:
category_sales[:5]

In [ ]:
category_sales[:5].index[0]


In [ ]:
online_sales_customers = pd.merge(online_sales, customers_data, on='고객ID')
online_sales_customers

In [ ]:
df_category_gender = pd.DataFrame()
for i in range(0, 5):
    category_name = category_sales[:5].index[i]
    df_category_gender[category_name]=(online_sales_customers[online_sales_customers['제품카테고리']==category_name].drop_duplicates(['고객ID'])['성별'].value_counts())
df_category_gender
    

In [ ]:
df_category_gender.plot(kind='bar')

In [ ]:
df_category_gender2 = pd.DataFrame()
for i in range(0, 5):
    category_name = category_sales[:5].index[i]
    df_category_gender2[category_name]=(online_sales_customers[online_sales_customers['제품카테고리']==category_name])['성별'].value_counts()
df_category_gender2
df_category_gender2.plot(kind='bar')

In [ ]:
## Evidence 1:  상위 판매 카테고리 5개 중 모든 카테고리 구매자는 여성이 남성보다 높다.

In [ ]:
for i in range(0,len(category_sales_gender)):
    category_name = category_sales_gender.loc[i]['제품카테고리']
    print(category_sales_gender.loc[i]['제품카테고리'])
    user_list = pd.merge(customers_data, online_sales[online_sales['제품카테고리']==category_name]['고객ID'].drop_duplicates().reset_index(drop=True), on=['고객ID'])
    user_list[user_list['성별'] == '여'].value_counts().sum()
    category_sales_gender.loc[i]['gender']



In [ ]:
user_list[user_list['성별'] == '남'].value_counts().sum()


In [ ]:



#marketing_spend= marketing_spend.assign(Month=pd.to_datetime(marketing_spend['날짜']).dt.to_period('M'))

#category_sales_gender['female'] = 
#category_sales_gender['male']  =

In [ ]:
online_sales['total_sales'] = online_sales['수량']*online_sales['평균금액']

In [ ]:
# 판매액이 높은 제품 카테고리순
order_high_purchase_category = online_sales.groupby('제품카테고리')['total_sales'].sum().sort_values(ascending=False)
order_high_purchase_category = order_high_purchase_category.reset_index()
order_high_purchase_category

In [ ]:
order_high_purchase_category[:5]

In [ ]:
#카테고리별 우수고객(구매금액 상위)

In [ ]:
prices_list = online_sales['배송료'].value_counts()
prices_list = prices_list.reset_index()
prices_list

In [ ]:
prices_list=prices_list[:100].sort_values('배송료')

In [ ]:
prices_list[:50]

In [ ]:

import matplotlib.pyplot as plt
prices_list.plot.bar(title='sdf', width=0.8)
plt.show()

In [ ]:
customers_data.head(13)

In [ ]:
individual_total_sales = online_sales.groupby('고객ID')['total_sales'].sum().reset_index().sort_values('total_sales',ascending=False)
individual_total_sales

In [ ]:

import matplotlib.pyplot as plt
y =  individual_total_sales['total_sales']
x =  individual_total_sales['고객ID']
fig = plt.figure()
plt.scatter(x,y)
fig.set_size_inches(30, fig.get_figheight(), forward=True)

plt.show()

In [ ]:
individual_total_sales['total_sales'].quantile([0.25, 0.5, 0.75,1])

In [ ]:
len(individual_total_sales[individual_total_sales['total_sales']>20000])

In [ ]:
highly_pay_customer_group =individual_total_sales[individual_total_sales['total_sales']>=20000]
target_customer_group = individual_total_sales[individual_total_sales['total_sales']<20000]

In [ ]:
distribute_4_cost = target_customer_group['total_sales'].quantile([0.25, 0.5, 0.75,1]).array
distribute_4_cost = np.array(distribute_4_cost).astype(int)
distribute_4_cost


In [ ]:
purchase_group = [
    target_customer_group[ (target_customer_group['total_sales'] < distribute_4_cost[0]) ],
    target_customer_group[ (target_customer_group['total_sales'] > distribute_4_cost[0]) & (target_customer_group['total_sales'] <=  distribute_4_cost[1]) ],
    target_customer_group[ (target_customer_group['total_sales'] > distribute_4_cost[1]) & (target_customer_group['total_sales'] <=  distribute_4_cost[2]) ] ,
    target_customer_group[ (target_customer_group['total_sales'] > distribute_4_cost[2])]
    
    ]

In [ ]:
purchase_group

In [ ]:
group_count = []
for i in range(4):
    group_count.append(len(purchase_group[i]))
group_count

In [ ]:
df = pd.DataFrame({"number of customers": group_count ,"cost_portion": distribute_4_cost})
df


In [ ]:
#distribute_4_cost
import matplotlib.pyplot as plt
df['number of customers'].plot.bar(title='number of customers in cost distribution', width=0.8)

plt.show()

In [ ]:
# 구매액(총 구매액)을 기준으로 백분위수를 구했을때 총 4개의 그룹이 발견되었고 4번째 그룹의 고객수는 20명이라 highly_pay_customer_group 로 별도 구분을 한다.
# 구매액이 상대적으로 적은 그룹과 highly_pay_customer_group의 사용자들과 무엇이 다른지 확인하고
# 각 그룹의 특징을 알아보자

In [ ]:
first_group_customers

In [ ]:
len(individual_total_sales[individual_total_sales['total_sales']>10000])

In [ ]:
sales_portion = individual_total_sales['평균금액'].quantile([0.25, 0.5, 0.75,1], interpolation='nearest').array

In [ ]:
sales_portion

In [ ]:
sales_portion[3]

In [ ]:
lowest_group_id = individual_total_sales.loc[ (individual_total_sales['평균금액']<sales_portion[0])]

In [ ]:
first_group_id = individual_total_sales.loc[ (individual_total_sales['평균금액']>sales_portion[0]) & (individual_total_sales['평균금액']<sales_portion[1] ) ]['고객ID']
first_group_id

In [ ]:
second_group_id = individual_total_sales.loc[ (individual_total_sales['평균금액']>sales_portion[1]) & (individual_total_sales['평균금액']<sales_portion[2] ) ]['고객ID']
second_group_id

In [ ]:
third_group_id = individual_total_sales[ individual_total_sales['평균금액']> sales_portion[2]]['고객ID']
third_group_id

In [ ]:
customers_data[ customers_data['고객ID'] == 'USER_0003'] 

In [ ]:
customers_data[ customers_data['고객ID'] == 'USER_1449'] 

In [ ]:
lowest_group_customers = pd.merge(customers_data, lowest_group_id, on='고객ID')

In [ ]:
first_group_customers = pd.merge(customers_data, first_group_id, on='고객ID')
first_group_customers['고객지역'].value_counts()

In [ ]:
second_group_customers = pd.merge(customers_data,second_group_id, on='고객ID')


In [ ]:
second_group_customers['가입기간'].mean()



In [ ]:
third_group_customers = pd.merge(customers_data,third_group_id, on='고객ID')
third_group_customers['고객지역'].value_counts()

In [ ]:
customers_data.groupby('고객지역')['가입기간'].mean()

In [ ]:
average_join_years = [lowest_group_customers['가입기간'].mean(), first_group_customers['가입기간'].mean(), second_group_customers['가입기간'].mean(), third_group_customers['가입기간'].mean()]
average_join_years

In [ ]:
average_join_years=np.array(average_join_years).astype(int)

In [ ]:
sales_portion

In [ ]:
import matplotlib.pyplot as plt

y = np.arange(4)
label = ['구매액_최저_그룹', '구매액2번째그룹', '구매액3번째그룹','구매액4번째그룹']
values = average_join_years


plt.plot(list(range(len(average_join_years))), average_join_years)
plt.show()

In [ ]:
index = y
plt.bar(label, values, color='red',alpha=0.5)

plt.show()

In [ ]:
sales_portion

In [ ]:

index = y
plt.bar(label, sales_portion, color='blue',alpha=0.5)

plt.show()

In [ ]:

len(lowest_group_customers[lowest_group_customers['성별']=='여'])

In [ ]:
tips_sum_by_day_male = [len(lowest_group_customers[lowest_group_customers['성별']=='남']),
                        len(first_group_customers[first_group_customers['성별']=='남']),
                        len(second_group_customers[second_group_customers['성별']=='남']),
                        len(third_group_customers[third_group_customers['성별']=='남'])]
tips_sum_by_day_female = [len(lowest_group_customers[lowest_group_customers['성별']=='여']),
                        len(first_group_customers[first_group_customers['성별']=='여']),
                        len(second_group_customers[second_group_customers['성별']=='여']),
                        len(third_group_customers[third_group_customers['성별']=='여'])
                        ]

In [ ]:
# Dodged Bar Chart (with same X coordinates side by side)
bar_width = 0.35
alpha = 0.5
p1 = plt.bar(index, tips_sum_by_day_male,bar_width,color='b',alpha=alpha,label='Male')
p2 = plt.bar(index + bar_width, tips_sum_by_day_female,
             bar_width,color='r',alpha=alpha,label='Female')
plt.title('Dodged Bar Chart of Sum of Tips by Day & Sex', fontsize=10)
plt.ylabel('Sum of Tips', fontsize=10)
plt.xlabel('Day', fontsize=10)
plt.xticks(index, label, fontsize=10)
plt.legend((p1[0], p2[0]), ('Male', 'Female'), fontsize=10)
plt.show()


In [ ]:
#evidence : 여성이 전체 구매비율이 남자보다 높지만, 남자는 구매액이 높을 수록 구매를 하는 경향이 있다. 남성고객의

In [ ]:
discount_coupon.head(3)

In [ ]:
marketing_spend.head(5)

In [ ]:
tax_amount.head(5)

### 데이터 관찰하기

불러온 데이터를 관찰해 봅시다. 대표적으로 onlines_sales를 확인해 보겠습니다. 

In [ ]:
online_sales.head(3)

### 시간에 따른 거래량 확인하기

시간에 따른 거래량을 시각화로 파악해보겠습니다. 

In [ ]:
# 'Transaction_Date' 컬럼을 날짜 타입으로 변환하고 월별로 집계
# 'Month' 컬럼으로 거래량 계산
counts_by_month = online_sales.assign(Month=pd.to_datetime(online_sales['거래날짜']).dt.to_period('M'))['Month'].value_counts().sort_index()

# 시간에 따른 거래량 선 그래프 그리기
plt.figure(figsize=(15, 6))
sns.lineplot(x=counts_by_month.index.astype(str), y=counts_by_month.values)
plt.title('시간에 따른 거래량')
plt.xlabel('월')
plt.ylabel('거래량')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 'Transaction_Date' 컬럼을 날짜 타입으로 변환하고 월별로 집계
# 'Day' 컬럼으로 거래량 계산
counts_by_day= online_sales.assign(Day=pd.to_datetime(online_sales['거래날짜']).dt.to_period('D'))['Day'].value_counts().sort_index()

# 날짜에 따른 거래량 선 그래프 그리기
plt.figure(figsize=(115, 6))
sns.lineplot(x=counts_by_day.index.astype(str), y=counts_by_day.values)
plt.title('날짜에 따른 거래량')
plt.xlabel('일')
plt.ylabel('거래량')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#위 차트에서 일정적으로 높고 낮음의 주기(Trend)가 있음

In [ ]:
day_sales = online_sales.assign(Day=pd.to_datetime(online_sales['거래날짜']).dt.to_period('D'))['Day'].value_counts()
day_sales = day_sales.reset_index()
day_sales


In [ ]:
day_sales['day_of_week'] = day_sales.Day.dt.day_of_week
day_sale= day_sales.groupby("day_of_week")['count'].sum()

# 날짜에 따른 거래량 선 그래프 그리기
plt.figure(figsize=(15, 6))
sns.lineplot(x=day_sale.index.astype(str), y=day_sale.values)
plt.title('요일에 따른 거래량')
plt.xlabel('요일')
plt.ylabel('거래량')
plt.xticks(rotation=45)
plt.show()

In [ ]:
day_sales['days_in_month'] = day_sales.Day.dt.day
day_sales2 = day_sales.groupby("days_in_month")['count'].sum()
# 날짜에 따른 거래량 선 그래프 그리기
plt.figure(figsize=(15, 6))
sns.lineplot(x=day_sales2.index.astype(str), y=day_sales2.values)
plt.title('날짜에 따른 거래량')
plt.xlabel('날짜')
plt.ylabel('거래량')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#가격에 따른 고객분류
#날짜에 따른 고객분류?


### 성별에 따른 고객 분류

고객을 성별에 따라 분류해보겠습니다. 

In [ ]:
gender_counts = customers_data['성별'].value_counts()

# 원그래프 그리기
plt.figure(figsize=(8, 8))  # 그래프 크기 설정
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)  # 원그래프 그리기
plt.title('성별 시각화')  # 그래프 제목 설정
plt.show()  # 그래프 보여주기

### 지역에 따른 고객 분류

고객을 지역에 따라 분류해보겠습니다. 

In [ ]:
# customers_data에서 '고객지역' 열을 사용하여 지역별 거래량 계산
location_counts = customers_data['고객지역'].value_counts()

# 바 그래프 그리기
plt.figure(figsize=(10, 6))  # 그래프 크기 설정
sns.barplot(x=location_counts.index, y=location_counts.values)  # seaborn을 사용한 바 그래프 그리기
plt.title('고객 지역 시각화')  # 그래프 제목 설정
plt.xlabel('고객지역')  # x축 레이블 설정
plt.xticks(rotation=45)  # x축 레이블 회전
plt.show()  # 그래프 보여주기

### RFM 고객 세분화 분석

이제 RFM 방법을 사용해서 고객을 세분화해보겠습니다. 

In [ ]:
# 원본 데이터셋 복사
rfm_online_sales = online_sales.copy()

# 날짜 형식 변환
rfm_online_sales['거래날짜'] = pd.to_datetime(rfm_online_sales['거래날짜'])

# 데이터 내 마지막 날짜 계산
last_date = rfm_online_sales['거래날짜'].max()

# Recency 계산
recency_data = rfm_online_sales.groupby('고객ID')['거래날짜'].max().reset_index()
recency_data['Recency'] = (last_date - recency_data['거래날짜']).dt.days

# Frequency 계산
frequency_data = rfm_online_sales.groupby('고객ID')['거래ID'].count().reset_index()
frequency_data.rename(columns={'거래ID': 'Frequency'}, inplace=True)

# Monetary 계산
rfm_online_sales['SalesValue'] = rfm_online_sales['수량'] * rfm_online_sales['평균금액']
monetary_data = rfm_online_sales.groupby('고객ID')['SalesValue'].sum().reset_index()
monetary_data.rename(columns={'SalesValue': 'Monetary'}, inplace=True)

# RFM 데이터 결합
rfm_data = recency_data.merge(frequency_data, on='고객ID').merge(monetary_data, on='고객ID')

# Recency, Frequency, Monetary에 점수 부여
rfm_data['R_Score'] = pd.qcut(rfm_data['Recency'], 4, labels=[4, 3, 2, 1])
rfm_data['F_Score'] = pd.qcut(rfm_data['Frequency'], 4, labels=[1, 2, 3, 4])
rfm_data['M_Score'] = pd.qcut(rfm_data['Monetary'], 4, labels=[1, 2, 3, 4])

# RFM 스코어 계산
rfm_data['RFM_Score'] = rfm_data['R_Score'].astype(str) + rfm_data['F_Score'].astype(str) + rfm_data['M_Score'].astype(str)


In [ ]:
# 고객 세그먼트 분류 함수
def classify_customer_segment(row):
    R, F, M = row['R_Score'], row['F_Score'], row['M_Score']
    
    if R == 4 and F == 4 and M == 4:
        return 'VIP고객'
    elif R >= 2 and F >= 3 and M == 4:
        return '충성고객'
    elif R >= 3 and F <= 3 and M <= 3:
        return '잠재충성고객'
    elif R == 2 and F < 2 and M < 2:
        return '신규고객'
    elif R >= 3 and F < 2 and M < 2:
        return '정체고객'
    elif R >= 3 and F >= 3 and M >= 3:
        return '관심필요고객'
    elif 2 <= R <= 3 and F < 3 and M < 3:
        return '잠드려는고객'
    elif R < 3 and 2 <= F <= 4 and 2 <= M <= 4:
        return '이탈우려고객'
    elif R < 2 and F == 4 and M == 4:
        return '놓치면안될고객'
    elif 2 <= R <= 3 and 2 <= F <= 3 and 2 <= M <= 3:
        return '겨울잠고객'
    elif R < 2 and F < 2 and M < 2:
        return '이탈고객'
    else:
        return '기타'
    
# rfm_data에 'Customer_Segment' 컬럼 추가
rfm_data['Customer_Segment'] = rfm_data.apply(classify_customer_segment, axis=1)


In [ ]:
# 가능한 모든 세그먼트 정의
all_segments = ['VIP고객', '충성고객', '잠재충성고객', '신규고객', '정체고객', '관심필요고객', 
                '잠드려는고객', '이탈우려고객', '놓치면안될고객', '겨울잠고객', '이탈고객', '기타']

# 각 세그먼트별 고객 수 계산
segment_counts = rfm_data['Customer_Segment'].value_counts()

# 모든 세그먼트에 대한 고객 수를 0으로 초기화하고, 계산된 값으로 업데이트
segment_counts_all = {segment: 0 for segment in all_segments}
segment_counts_all.update(segment_counts)

# 데이터프레임으로 변환
segment_counts_df = pd.DataFrame(list(segment_counts_all.items()), columns=['Segment', 'Count'])

# 세그먼트별 고객 수를 내림차순으로 정렬
segment_counts_sorted_df = segment_counts_df.sort_values(by='Count', ascending=False)

# 바 그래프 시각화
plt.figure(figsize=(15, 8))
plt.bar(segment_counts_sorted_df['Segment'], segment_counts_sorted_df['Count'], color='skyblue')
plt.title('고객 세그먼트별 분포')
plt.xlabel('고객 세그먼트')
plt.ylabel('고객 수')
plt.xticks(rotation=45)
plt.show()